### Feature extraction from a real file

In [22]:
import numpy as np 
import pandas as pd

In [23]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv', sep='\t')

In [24]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [25]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [26]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X = df['message']

In [29]:
y = df['label']

In [30]:
# creating train and test suit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [31]:
# Transform documents into feature vectors
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
count_vect = CountVectorizer() # this module allows us to do all the transformations

In [33]:
X # we will pass this to the count vectorizer and transform it.

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

### Two steps to transform with the Count Vectorizer

1-FIT VECTORIZER TO THE DATA (build a vocab, count the number of words...)

2-TRANSFORM THE ORIGINAL TEXT MESSAGE --> VECTOR

In [34]:
# OPTION 1: Two steps (fit, transform)
# count_vect.fit(X_train) # build vocab, count nuber of words
# X_train_counts = count_vect.transform(X_train) # transform the text

# OPTION 2: All in one step (fit and transform)
X_train_counts = count_vect.fit_transform(X_train) # build vocab, count words and transforms

In [35]:
# this is a huge sparse matrix
X_train_counts # 3733 messages x 7082 unique words in those messages

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [36]:
X_train.shape

(3733,)

In [37]:
# we have the messages and the vocab count (see last step in 01-Feature-Extraction-from-Text2.ipynb)
X_train_counts.shape 

(3733, 7082)

In [39]:
# Part 2: Tfid Transformation and Clasification
from sklearn.feature_extraction.text import TfidfTransformer

In [41]:
tfidf_transformer = TfidfTransformer()

In [42]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [43]:
X_train_tfidf.shape # not just counter: term frequency * inverse document's frequency

(3733, 7082)

In [44]:
# Counter Vectorization and Tfid Transformation into one step
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
vectorizer = TfidfVectorizer()

In [46]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [47]:
# Step 3. Clasification: train a clasifier
from sklearn.svm import LinearSVC

In [48]:
clf = LinearSVC()

In [49]:
clf.fit(X_train_tfidf, y_train) # our traininig set has been vectorized into a full vocabulary

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

### To perform an analysis in our test set, we have to do all the same procedures as the training set

In [50]:
# Perform Vectozization, Tfid Transformation and Clasification in one single pipeline
from sklearn.pipeline import Pipeline

In [51]:
# pipeline with Vectorization, Tfid Transformation and Clasification
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())]) 

In [52]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [53]:
predictions = text_clf.predict(X_test) # creating our model with all the training data

In [54]:
from sklearn.metrics import confusion_matrix, classification_report

In [55]:
print(confusion_matrix(y_test, predictions)) # very good results

[[1586    7]
 [  12  234]]


In [56]:
print(classification_report(y_test, predictions)) # very good results

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [57]:
from sklearn import metrics

In [58]:
metrics.accuracy_score(y_test, predictions) # 99% score

0.989668297988037

In [60]:
# using our successed model to predict another message
text_clf.predict(["Congratinations! You've been selected as a winner. TEXT WON to 44255 congratulations free entry to contest."])

array(['spam'], dtype=object)

In [61]:
text_clf.predict(["Dear Marty, how are you?. We are going to do ski this weekend. Would you like to join us?"])

array(['ham'], dtype=object)